# Code to work with the website

This notebook showcases all processed with the website that Python can do. You need a Firebase Admin key to work with the Firestore database.

## Create array for the website of evaluation books

In [ ]:
import pandas as pd

In [ ]:
evaluation_books = pd.read_csv("../data/external/evaluation.csv")

In [ ]:
textfile = open("../data/interim/evaluation.txt", "w")
for element in evaluation_books.itemID.tolist():
    textfile.write(str(element) + ",")
textfile.close()

## Download Decision data

In [ ]:
import pandas as pd
from src.data.firestore import connect_to_firestore, retrieve_documents_from_collection

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
decisions_dict = retrieve_documents_from_collection(firestore_client, collection="decisions")

In [ ]:
decisions = pd.DataFrame(decisions_dict)

In [ ]:
decisions.to_csv("../data/processed/decisions.csv", index = False)

## Upload book information to the website

In [ ]:
import pandas as pd
from src.data.firestore import connect_to_firestore, write_documents_to_collection

In [ ]:
books_path = "../data/processed/books.csv"
books_reduced_path = "../data/processed/books_reduced.csv"

In [ ]:
books = pd.read_csv(books_path,index_col = 0)
books_reduced_index = pd.read_csv(books_reduced_path,index_col = 0)

In [ ]:
books.set_index("itemID", inplace = True)

In [ ]:
books_reduced = books.filter(books_reduced_index.book_id.tolist(), axis = 0)

In [ ]:
books_reduced.reset_index(inplace = True)

In [ ]:
books_reduced

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

Since we have a free tier on firebase we need to upload multiple days. In moment we have the first 10.000 online.

In [ ]:
write_documents_to_collection(firestore_client, books_reduced, collection="books")

## Upload a new model to the website

If a team sends us a new model, we can upload it to the website with a simple function. Before we upload the recommendations, we have to create document in the model_scores collection for the new model. This requires to use the firebase backend.

In [1]:
import pandas as pd
from src.data.firestore import connect_to_firestore, write_documents_to_collection

Code for Team DataMinerz

In [ ]:
DataMinerz_path = "../data/processed/rule_based_dataminerz.csv"

In [ ]:
DataMinerz_recommendations = pd.read_csv(DataMinerz_path,index_col = 0)

In [ ]:
DataMinerz_recommendations

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, DataMinerz_recommendations, collection="recommendations")

Code for Team MannheimMiners

In [ ]:
mannheimminers_path = "../data/processed/MannheimMinersModel3.csv"

In [ ]:
mannheimminers_recommendations = pd.read_csv(mannheimminers_path,index_col = 0)

In [ ]:
mannheimminers_recommendations.reset_index(inplace = True)

In [ ]:
mannheimminers_recommendations.dropna(inplace=True)

In [ ]:
mannheimminers_processed = mannheimminers_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
mannheimminers_processed.replace({"model_id":{"3":"third"}}, inplace = True)

In [ ]:
mannheimminers_processed

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, mannheimminers_processed, collection="recommendations")

Code for Team team404

In [ ]:
team404_path = "../data/processed/team404_2.xlsx"

In [ ]:
team404_recommendations = pd.read_excel(team404_path,index_col = 0)

In [ ]:
team404_recommendations.dropna(inplace=True)

In [ ]:
team404_recommendations.reset_index(inplace = True)

In [ ]:
team404_processed = team404_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
team404_processed

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, team404_processed, collection="recommendations")

Code for Team minefromhome

In [ ]:
minefromhome_path = "../data/processed/Team_05.csv"

In [ ]:
minefromhome_recommendations = pd.read_csv(minefromhome_path,index_col = 0, names = ["book_id",
                                                                                     "model_id",
                                                                                     "team_id",
                                                                                     "recommendation_1",
                                                                                     "recommendation_2",
                                                                                     "recommendation_3",
                                                                                     "recommendation_4",
                                                                                     "recommendation_5"])

In [ ]:
minefromhome_recommendations.dropna(inplace=True)

In [ ]:
minefromhome_recommendations.reset_index(inplace = True)

In [ ]:
minefromhome_processed = minefromhome_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
minefromhome_processed

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, minefromhome_processed, collection="recommendations")

Code for Team flying_chicken_wings

In [ ]:
flying_chicken_wings_path = "../data/processed/Team6_result_2ndModel.xlsx"

In [ ]:
flying_chicken_wings_recommendations = pd.read_excel(flying_chicken_wings_path)

In [ ]:
flying_chicken_wings_recommendations.dropna(inplace=True)

In [ ]:
flying_chicken_wings_processed = flying_chicken_wings_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
flying_chicken_wings_processed

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, flying_chicken_wings_processed, collection="recommendations")

Code for d2g2

In [ ]:
d2g2_path = "../data/processed/d2g2_results_2.xlsx"

In [ ]:
d2g2_recommendations = pd.read_excel(d2g2_path)

In [ ]:
d2g2_recommendations

In [ ]:
d2g2_recommendations.dropna(inplace=True)

In [ ]:
d2g2_processed = d2g2_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
d2g2_processed["team_id"] = "d2g2"

In [ ]:
d2g2_processed

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, d2g2_processed, collection="recommendations")

Code for dataworms

In [2]:
Dataworms_path = "../data/processed/final_result_Dataworms.csv"

In [3]:
Dataworms_recommendations = pd.read_csv(Dataworms_path, 
                                   dtype={'recommendation_1': str, 'recommendation_2': str,
                                          'recommendation_3': str, 'recommendation_4': str, 
                                          'recommendation_5': str})

In [4]:
Dataworms_recommendations

,book_id,model_id,team_id,recommendation_1,recommendation_2,recommendation_3,recommendation_4,recommendation_5
0,12,Hybrid_RS,the_dataworms,38044,15978,18081,37565,27058
1,45274,Hybrid_RS,the_dataworms,1746,63413,7262,6210,62473
2,10104,Hybrid_RS,the_dataworms,10143,41762,71798,20416,40979
3,41371,Hybrid_RS,the_dataworms,70558,64698,64050,72736,58150
4,14015,Hybrid_RS,the_dataworms,13103,47952,19563,66672,21188
...,...,...,...,...,...,...,...,...
995,78384,Hybrid_RS,the_dataworms,56354,72389,22154,40979,71593
996,68157,Hybrid_RS,the_dataworms,21470,19503,66167,55547,58623
997,44964,Hybrid_RS,the_dataworms,75198,25989,68320,70107,23343
998,12971,Hybrid_RS,the_dataworms,32816,15800,49248,56107,21904


In [5]:
Dataworms_recommendations.dropna(inplace=True)

In [6]:
Dataworms_processed = Dataworms_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [7]:
Dataworms_processed

,book_id,model_id,team_id,recommendation_1,recommendation_2,recommendation_3,recommendation_4,recommendation_5
0,12,Hybrid_RS,the_dataworms,38044,15978,18081,37565,27058
1,45274,Hybrid_RS,the_dataworms,1746,63413,7262,6210,62473
2,10104,Hybrid_RS,the_dataworms,10143,41762,71798,20416,40979
3,41371,Hybrid_RS,the_dataworms,70558,64698,64050,72736,58150
4,14015,Hybrid_RS,the_dataworms,13103,47952,19563,66672,21188
...,...,...,...,...,...,...,...,...
995,78384,Hybrid_RS,the_dataworms,56354,72389,22154,40979,71593
996,68157,Hybrid_RS,the_dataworms,21470,19503,66167,55547,58623
997,44964,Hybrid_RS,the_dataworms,75198,25989,68320,70107,23343
998,12971,Hybrid_RS,the_dataworms,32816,15800,49248,56107,21904


In [8]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [9]:
write_documents_to_collection(firestore_client, Dataworms_processed, collection="recommendations")

100%|██████████| 998/998 [03:25<00:00,  4.85it/s]


## Update book information to the website

In [10]:
import pandas as pd
import pickle
from src.data.firestore import connect_to_firestore, write_documents_to_collection

In [11]:
books_path = "../data/processed/books.csv"
books_reduced_path = "../data/processed/books_reduced.pkl"

In [12]:
books = pd.read_csv(books_path,index_col = 0)
books_reduced_index = pd.read_pickle(books_reduced_path)

In [13]:
books_added_index = set(pd.melt(Dataworms_processed, value_vars=['recommendation_1',
                                       'recommendation_2',
                                       'recommendation_3',
                                       'recommendation_4',
                                       'recommendation_5']).value.tolist())

In [14]:
len(books_reduced_index)

25621

In [15]:
len(books_added_index)

3359

In [16]:
books_reduced_added_index = {index for index in books_added_index if index not in books_reduced_index}

In [17]:
len(books_reduced_added_index)

770

In [18]:
books.set_index("itemID", inplace = True)

In [19]:
books_reduced_added = books.filter(books_reduced_added_index, axis = 0)

In [20]:
books_reduced_added.reset_index(inplace = True)

In [21]:
books_reduced_added

,itemID,title,author,publisher,category_names,cover_url
0,12289,Master Bun the Bakers' Boy,Allan Ahlberg,Penguin Random House Children's UK,Children’s / Teenage fiction: General fiction,https://assets.thalia.media/img/artikel/7d7bc8...
1,36866,The Wheels -The Friendship Race (Turkish Edition),"Kidkiddos Books, Inna Nusinsky",KIDKIDDOS BOOKS LTD,Early years: time and seasons,https://assets.thalia.media/img/artikel/b7f5b7...
2,73731,There Are No Ants in My Pants! for Boys Only(r),Penelope Dyan,Bellissima Publishing LLC,Early years: numbers and counting,https://assets.thalia.media/img/artikel/4c73c5...
3,40964,The Crayola Counting Book,Mari C. Schuh,LERNER PUBN,Early years: numbers and counting,https://assets.thalia.media/img/artikel/9437bb...
4,4101,Cohasset from A to Z,Gael Daly,Converpage,Early years: letters and words,https://assets.thalia.media/img/artikel/5da861...
...,...,...,...,...,...,...
765,73712,The Muddlemoor Mysteries: Peril at the Bake Off,Ruth Quayle,Andersen Press Ltd,Children’s / Teenage fiction: Crime and myster...,https://assets.thalia.media/img/artikel/5f6e6f...
766,67569,Charlottas fantastische Reise nach Flüsterleise,Frank Didden,epubli,Science fiction,https://assets.thalia.media/img/artikel/5d4625...
767,8181,The Mermaid's Dolphin,Coral Ripley,Hachette Children's Group,Children’s / Teenage fiction: Fantasy,https://assets.thalia.media/img/artikel/00c12a...
768,67575,Im Zug,"Steve Martin, Putri Febriana",Ars Edition GmbH,Children’s interactive and activity books and ...,https://assets.thalia.media/img/artikel/68c2fd...


In [22]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [23]:
write_documents_to_collection(firestore_client, books_reduced_added, collection="books")

100%|██████████| 770/770 [01:45<00:00,  7.29it/s]


In [24]:
len(books_reduced_index.union(books_reduced_added_index))

26391

In [25]:
with open('../data/processed/books_reduced.pkl', 'wb') as file:
    pickle.dump(books_reduced_index.union(books_reduced_added_index), file)

## Delete error data

In [ ]:
from src.data.firestore import connect_to_firestore, write_documents_to_collection

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
batch = firestore_client.batch()

doc_ref = firestore_client.collection('model_scores').where('score', '!=', False).get()

for doc in doc_ref:
       batch.delete(doc.reference)

batch.commit()

In [ ]:
float('nan')